In [1]:
# Iš duomenų aplanko, kuriame yra atsisiųsti įrašai iš Zive serverio,
# atrnkami duomenys iš dviejų Excel sąrašų: vienas su rpeak sutampančiais,
# kitas - su nesutampančiais. Atrenkami tik tie įrašai, prie kurių pažymėta 
# įrašo kokybė. Taip pat pasiimama informacija apie userId, recordingId,
# comments. Tuo pagrindu suformuojama papildoma informacija faile comments_new.csv.
# Skriptas atrinktus duomenis ir suformuotą comments_new.csv perrašo į aplanką rec_dir_target  
# Šie duomenys naudojami Makete list.json formavimui ir darbui su duomenimis.

import pandas as pd
import numpy as np
import json, sys
from pathlib import Path
import shutil


my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)


# Vietinės talpyklos aplankas
db_folder = 'DUOMENU_TVARKYMAS_2023'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais ir anotacijomis (.json), iš kurio skaitomi atrinkti įrašai
rec_dir_source = Path(db_path, 'DUOMENYS_IS_SERVERIO_2023_03_26')

# Nuoroda į aplanką su EKG įrašais ir anotacijomis (.json), į kurį perrašomi atrinkti įrašai
rec_dir_target = Path(db_path, 'ATRINKTI_DUOMENYS')


# -------------------  ĮRAŠAI SU SUTAMPANČIAIS RPEAKS ---------------------------------------
file_path = Path(db_path, '1-Zilvino_130_Irasu_sarasas_kokybes_zymejimui_2023-03-20.xlsx')
print("\nSąrašas su sutampančiais rpeaks.")
print("\nFailas: ", file_path)

# read list
df_tmp = pd.read_excel(file_path)
# print(df_tmp.head(1))

# select columns
columns_to_select = ['filename', 'userId', 'recId', 'nesutmp', 'quality', 'comment']
df_st = df_tmp.loc[:, columns_to_select]

# replace empty values with NaN
df_st['quality'] = pd.to_numeric(df_st['quality'], errors='coerce')
# drop rows with missing values in the 'quality' column
df_st = df_st.dropna(subset=['quality'])
# replace missing values with None
df_st['comment'] = df_st['comment'].replace('', None)

# specify desired data types
dtype = {'filename': 'string', 'userId': 'string', 'recId': 'string', 'nesutmp': 'int', 'quality':'int', 'comment': 'string'}
# convert data types of columns
df_st = df_st.astype(dtype)

print(df_st.dtypes)
print(df_st.head(10))

# -------------------  ĮRAŠAI SU NESUTAMPANČIAIS RPEAKS ---------------------------------------
file_path = Path(db_path, 'rezultatu_pasiskirstymas_per_irasus_plius_atranka_2023_02_10.xlsx')
print("\nSąrašas su nesutampančiais rpeaks.")
print("\nFailas: ", file_path)

# read list
df_tmp = pd.read_excel(file_path)
# print(df_tmp.head(1))

# select columns
columns_to_select = ['filename', 'userId', 'recId', 'nesutmp', 'quality', 'comment']
df_ns = df_tmp.loc[:, columns_to_select]

# replace empty values with NaN
df_ns['quality'] = pd.to_numeric(df_ns['quality'], errors='coerce')
# drop rows with missing values in the 'quality' column
df_ns = df_ns.dropna(subset=['quality'])
# replace missing values with None
df_ns['comment'] = df_ns['comment'].replace('', None)

# specify desired data types
dtype = {'filename': 'string', 'userId': 'string', 'recId': 'string', 'nesutmp': 'int', 'quality':'int', 'comment': 'string'}
# convert data types of columns
df_ns = df_ns.astype(dtype)

print(df_ns.dtypes)
print(df_ns.head(10))

# -------------------  MIT2ZIVE ĮRAŠAI --------------------------------------------------------
file_path = Path(db_path, 'mit2zive_irasai_komentarai.xlsx')
print("\nSąrašas su nesutampančiais rpeaks.")
print("\nFailas: ", file_path)

# read list
df_tmp = pd.read_excel(file_path)
# print(df_tmp.head(1))

# select columns
columns_to_select = ['filename', 'userId', 'recId', 'nesutmp', 'quality', 'comment']
df_mit2zive = df_tmp.loc[:, columns_to_select]

# replace empty values with NaN
df_mit2zive['quality'] = pd.to_numeric(df_mit2zive['quality'], errors='coerce')
# drop rows with missing values in the 'quality' column
df_mit2zive = df_mit2zive.dropna(subset=['quality'])
# replace missing values with None
df_mit2zive['comment'] = df_mit2zive['comment'].replace('', None)

# specify desired data types
dtype = {'filename': 'string', 'userId': 'string', 'recId': 'string', 'nesutmp': 'int', 'quality':'int', 'comment': 'string'}
# convert data types of columns
df_mit2zive = df_mit2zive.astype(dtype)

print(df_mit2zive.dtypes)
print(df_mit2zive.head(10))

# ------------------- VISŲ TRIJŲ APJUNGIMAS ---------------------------------------

# Apjungiame df_st ir df_ns į viena dataframe
concatenated_df = pd.concat([df_st, df_ns])

# Surandame abiejuose sąrašuose dublius, atsispausdiname ir panaikiname
duplicates = concatenated_df[concatenated_df.duplicated('filename')]
print('duplicates:')
print(duplicates)

# print df_ns rows with dublicated filenames
for _, duplicate in duplicates.iterrows():
    print(duplicate.iloc[0])
    row_ns = df_ns[df_ns['filename'] == duplicate.iloc[0]]
    print("kestucio:",row_ns)
    row_st = df_st[df_st['filename'] == duplicate.iloc[0]]
    print("zilvino:",row_st)


concatenated_df = pd.concat([concatenated_df, df_mit2zive])

# Perrašome tinkamus įrašus į atrinktų duomenų aplanką

for i in range(len(concatenated_df)):
    filename = concatenated_df.iloc[i]['filename']
    number_float = float(filename)
    filename = '{:.3f}'.format(number_float)
    # concatenated_df.iloc[i]['filename'] = filename
    print(filename)
    
    # įrašas
    # file_path = Path(rec_dir_source, filename)
    # new_file_path = Path(rec_dir_target, filename)
    # shutil.copyfile(file_path, new_file_path)
    # # json
    # file_path = Path(rec_dir_source, filename + '.json')
    # new_file_path = Path(rec_dir_target, filename + '.json')
    # shutil.copyfile(file_path, new_file_path)

    print(f'{i+1} of {len(concatenated_df)}')

# Paverčiame datafreimą concatenated_df į analogišką masyvui MAKETE
# ir įrašome į comments_new.csv
# pvz.
# file_name","incl","flag","comment"
# 1626934.963,1,1,"Albertas, gale atsij. elektr."
# 1626931.201,1,0,"Ignas, raumen."

# remove in  concatenated_df columns userId, recId
concatenated_df = concatenated_df.drop(['userId', 'recId'], axis=1)
 
# change columns names in concatenated_df filename, nesutmp, quality into file_name, incl, flag and write
concatenated_df.rename(columns={'filename': "file_name", 'nesutmp': 'incl', 'quality': 'flag', 'comment': 'comment'}, inplace=True)
# together with names of columns into csv

# write the concatenated dataframe to a new file
file_name = 'comments_new.csv'
file_path = Path(rec_dir_target, file_name)
concatenated_df.to_csv(file_path, index=False)
print("Sąrašas įrašytas į: ", file_path)



OS in my system :  linux

Sąrašas su sutampančiais rpeaks.

Failas:  /home/kesju/DI/DUOMENU_TVARKYMAS_2023/1-Zilvino_130_Irasu_sarasas_kokybes_zymejimui_2023-03-20.xlsx
filename    string
userId      string
recId       string
nesutmp      int64
quality      int64
comment     string
dtype: object
      filename                    userId                     recId  nesutmp  quality                                            comment
0  1626934.963  60a917b354352a3df86dc1f2  60f9188487cf6632713780c5        0        0  Kokybė puiki, išskyrus pabaigą, būtina pažymėt...
1  1626931.201  60a917b354352a3df86dc1f2  60f9188487cf66240e3780bf        0        0  Neaišku, kas yra ties 12300 klausimas gydytoju...
2  1630715.664  613b1bc63d08d41309cdc8f1  613f57ea3d08d4628bcdcc7a        0        0  Pageidautina užblokuoti 2 pūpsnius ties 15600,...
3  1630714.569  613b1c013d08d44862cdc8f2  613f59083d08d40605cdce2a        0        0                                               <NA>
4  1630729.576  613b1c0